# DAT 19: Homework 4 Assignment - SVMs, Trees, RF

## Instructions

The goal of this homework is to review and bring together what we have learned about Support Vector Machines, Decision Trees, Random Forests, and ensembles. 

Please do all your analysis to answer the questions below in this Jupyter notebook. Show your work.

**Please submit your completed notebook by 6:30PM on Wednesday, February 17.**

## About the Data

Use the cancer_uci.csv dataset in the Data directory of our course repo. This is the [Breast Cancer Wisconsin](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Original)) dataset from the UCI ML Repository.

## Homework Assignment

**1) Load the data and check for balance between the two classes. If the ratio is less than 60/40 rebalance the classes to 50/50. We've provided some help here to get you started.**

In [ ]:
import pandas as pd
import numpy as np

In [3]:

canc = pd.read_csv("../data/cancer_uci.csv", index_col=0)
canc.head()
canc.Class.value_counts()
# What are the frequencies of each class?

Benign       458
Malignant    241
Name: Class, dtype: int64

We have imbalanced classes so we need to decide if we want to undersample, and take only 241 values from the Benign category, or oversample, and artificially inflate the volume of malignant data. First, let's convert to binary 1,0 for classification.

In [4]:
canc.Class = canc.Class.map({'Benign':0,'Malignant':1})
canc.Class.value_counts()

0    458
1    241
Name: Class, dtype: int64

To undersample, we would throw away almost half of our benign examples, which would greatly alter our dataset and we don't want to lose that much info. So let's oversample! Here is a pattern for how to oversample:

In [66]:
# Separate your two classes:
mal_example = canc[canc.Class == 1]
benign_example = canc[canc.Class == 0]

# Oversample the malignant class to have a 50/50 ratio:
mal_over_example = mal_example.sample(458,replace=True)

# Recombine the two frames:
over_sample = pd.concat([mal_over_example,benign_example])

# Sanity check the length:

print len(over_sample)

916


**2) Are the features normalized? If not, use the scikit-learn standard scaler to normalize them.**

In [28]:
over_sample.isnull().values.any()

False

In [192]:
X_train=over_sample[over_sample.columns[0:-1]]
y_train=over_sample[over_sample.columns[[-1]]]

In [193]:
X_train['Bare_Nuclei'].value_counts()

1     420
10    245
8      42
2      42
5      41
3      38
4      37
?      20
9      15
7      13
6       3
Name: Bare_Nuclei, dtype: int64

In [194]:
# X_train['Bare_Nuclei']=pd.to_numeric(X_train['Bare_Nuclei'], errors='coerce')
X_train.convert_objects(convert_numeric=True)
# X_train['Bare_Nuclei']=X_train['Bare_Nuclei'].astype('int64')
X_train=X_train.replace('?', 0)
X_train['Bare_Nuclei'].value_counts()



C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


1     420
10    245
8      42
2      42
5      41
3      38
4      37
0      20
9      15
7      13
6       3
Name: Bare_Nuclei, dtype: int64

In [195]:
X_train=X_train.fillna(0)

**3) Train a linear SVM, using the cross validated accuracy as the score (use the scikit-learn method).**

In [196]:
# print X_train.describe()
# print y_train.describe()


In [197]:
#your code here
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


clf=SVC()
clf.fit(X_train, y_train)
pred=clf.predict(X_train)
accuracy_score(y_train, pred)


C:\Anaconda2\lib\site-packages\sklearn\svm\base.py:472: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


1.0

**4) Display the confusion matrix, classification report, and AUC.**

In [198]:
#your code here
from sklearn.metrics import confusion_matrix
from  sklearn.metrics import classification_report
confusion_matrix(y_train, pred)
print classification_report(y_train, pred)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       458
          1       1.00      1.00      1.00       458

avg / total       1.00      1.00      1.00       916



**5) Repeat steps 2 through 4 using a Decision Tree model. Are the results better or worse than the SVM?**

In [202]:
#your code here

from sklearn.tree import DecisionTreeRegressor
clf2= DecisionTreeRegressor(random_state=1)
clf2.fit(X_train, y_train)
pred2=clf2.predict(X_train)
accuracy_score(y_train,pred2)
print 'Confusion matrix for Decision Trees'
print confusion_matrix(y_train,pred2)
print
print 'Classification report for Decision Trees'
print classification_report(y_train, pred2)
print 'Results are the same as SVM'

Confusion matrix for Decision Trees
[[458   0]
 [  0 458]]

Classification report for Decision Trees
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       458
          1       1.00      1.00      1.00       458

avg / total       1.00      1.00      1.00       916

Results are the same as SVM


**6) Repeat steps 2 through 4 using a Random Forest model. Are the results better or worse than the SVM?**

In [201]:
#your code here
from sklearn.ensemble import RandomForestClassifier
clf3= RandomForestClassifier()
clf3.fit(X_train, y_train)
pred3=clf3.predict(X_train)
print 'accuracy score'
print accuracy_score(y_train,pred3)
print 'Confusion matrix for Decision Trees'
print confusion_matrix(y_train,pred3)
print
print 'Classification report for Decision Trees'
print classification_report(y_train, pred3)
print 'Results are a little worse than SVM'

accuracy score
0.998908296943
Confusion matrix for Decision Trees
[[458   0]
 [  1 457]]

Classification report for Decision Trees
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       458
          1       1.00      1.00      1.00       458

avg / total       1.00      1.00      1.00       916

Results are the same


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


### Extra Credit Questions
**The following questions are strongly encouraged, but not required for this homework assignment.**

**7) Combine the SVM and the Decision Tree model using the [Voting Classifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html). Are the results better than either of these base classifiers alone?**

In [ ]:
#your code here

**8) Train an SVM using the RBF kernel. Is this model better or worse?**

In [ ]:
#your code here